In [4]:
import pandas as pd 


## Reading complex(nested) Json files into Pandas DataFrames

 It is very common to have json data that is nested and this process can be very painful! 

 Lucky for us Pandas comes with a function called json_normalize, this will help us with flattening our nested json.
 
 #### Let us start with a simple json read. 

In [58]:

data = read_json('DataSources\my_json.json')
data

# [
#     {
#         "Name": "Adrian",
#         "Age": 33
#     },
#     {
#         "Name": "Julia",
#         "Age": 21
#     }
# ]

,Name,Age
0,Adrian,33
1,Julia,21


In [71]:
# Let`s get dirty with a nested json 
# import flatten_json

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

my_nested_json = {
    'Name':'John',
    'Location':
          {'City':'Los Angeles',
           'State':['CA','FL']},
    'hobbies':[
        'Music',
        'Running'
    ]
}

data = flatten_json(my_nested_json)
pd.json_normalize(data)


,Name,Location_City,Location_State_0,Location_State_1,hobbies_0,hobbies_1
0,John,Los Angeles,CA,FL,Music,Running


In [76]:
import json 
import pandas as pd 

#load json object
with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\raw_nyc_phil.json') as f:
    d = json.load(f)

#lets put the data into a pandas df
#clicking on raw_nyc_phil.json under "Input Files"
#tells us parent node is 'programs'
nycphil = pd.json_normalize(d['programs'])
nycphil.head(3)

,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37


## json_normalize() method comes with this signaure 
```
pd.json_normalize(
    data: Union[Dict, List[Dict]],
    record_path: Union[str, List, NoneType] = None,
    meta: Union[str, List[Union[str, List[str]]], NoneType] = None,
    meta_prefix: Union[str, NoneType] = None,
    record_prefix: Union[str, NoneType] = None,
    errors: Union[str, NoneType] = 'raise',
    sep: str = '.',
    max_level: Union[int, NoneType] = None,
)
```


#### Let us go thru some examples from simple to more complex nested json files.


In [131]:
import json 

with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\nested\nested_01.json') as f:
    data = json.load(f)

## Preatty print the json body
# print(dumps(my_data ,indent=2))

# {
#   "students":[
#     {
#       "student_id":1,
#       "student_name":"Adrian",
#       "course":[
#         "Math",
#         "Chemistry"
#       ]
#     },
#     {
#       "student_id":2,
#       "student_name":"Jim",
#       "course":[
#         "Math",
#         "Sports"
#       ]
#     }
#   ]
# }

## Use json_normalize() with default values
pd.json_normalize(data)
## we can see that this is not looking good 


## So looking at the json we can see that the parent node is "students"
## Let`s try to normalize using the node

## we can write it like this 
pd.json_normalize(data['students'])

## or using the record_path parameter
pd.json_normalize(data, record_path=['students'])


## Hmm still not ok, but we are making progress ! We are able to see the nested column 'course'
## Let`s unpack the course column as a standalone DF and append the parent metadata to it.

my_students = pd.json_normalize(data=data['students'] , record_path='course',meta=['student_id', 'student_name'])
my_students
## So let`s explain 

# 1 - We noticed that we have nested column 'course' and we managed to get some flat column 'student_id', 'student_name'
# 2 - We created a second DF now using the object data['students'] and drilling into the rcord_path='course' and finally
# we keept the flattned columns from our parent object data['students'].


,0,student_id,student_name
0,Math,1,Adrian
1,Chemistry,1,Adrian
2,Math,2,Jim
3,Sports,2,Jim


## Let us do anothe examples this time and use a more complex nested json



In [273]:
import json 


with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\nested\nested_02.json') as f:
    my_data = json.load(f)

# print(dumps(data , indent=2)) 

# {
#   "students":[
#     {
#       "student_id":1,
#       "student_name":"Adrian",
#       "course":[
#         "Math",
#         "Chemistry"
#       ],
#       "contact":[
#         "012345",
#         "54321"
#       ]
#     },
#     {
#       "student_id":2,
#       "student_name":"Jim",
#       "course":[
#         "Math",
#         "Sports"
#       ],
#       "contact":[
#         "999999",
#         "88888"
#       ]
#     }
#   ]
# }


## We can see we have multiple nested columns

## Let`s push this into json_normalize and see what comes out
pd.json_normalize(my_data['students'])


## ok, by default he will flattned the all non nested column under the "students" node.

## Let`s fix the other two nested columns

my_course = pd.json_normalize(my_data['students'], record_path=['course'], meta=['student_id','student_name'])
my_contact = pd.json_normalize(my_data['students'], record_path=['contact'], meta=['student_id','student_name'])

my_students = pd.merge(my_course, my_contact, how='inner', on=['student_id','student_name'])
my_students

,0_x,student_id,student_name,0_y
0,Math,1,Adrian,12345
1,Math,1,Adrian,54321
2,Chemistry,1,Adrian,12345
3,Chemistry,1,Adrian,54321
4,Math,2,Jim,999999
5,Math,2,Jim,88888
6,Sports,2,Jim,999999
7,Sports,2,Jim,88888


## Let us use different type of json setup and see how json_normalizer reacts 

In [272]:


import json 

with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\nested\nested_03.json') as f:
    my_data = json.load(f)

# print(dumps(my_data , indent=2)) 
# {
#   "students":[
#     {
#       "student_id":"1",
#       "student_name":"Adrian",
#       "course":[
#         "Math",
#         "Chemistry"
#       ],
#       "contact":{
#         "mobile":12345,
#         "home":54321
#       }
#     },
#     {
#       "student_id":"2",
#       "student_name":"Jim",
#       "course":[
#         "Math",
#         "Sports"
#       ],
#       "contact":{
#         "mobile":999999,
#         "home":88888
#       }
#     }
#   ]
# }




## ok, by default he will flattned the all non nested column under the "students" node.
## this time we see that we have changed our json "contact" node from an array [] into a list of objects {}
## this changed the dehaiviour of the json_normalize() method

pd.json_normalize(my_data['students'])

## we see that he already flatten the contact columns ! nice 

## but we still need to deal with the course nested column 

## Let`s fix it 

# create a DF from our initial flatten structure and then join
my_flat_1 = pd.json_normalize(my_data['students'])

# remove the nested column 
del my_flat_1['course']

# See the flatten 
my_flat_1


# Let`s create a df with the course and the student_id (we will this to join our DF`s)
my_flat_courses = pd.json_normalize(my_data['students'], record_path=['course'], meta=['student_id'])

## create the final DF by merging the two DF`s on the 'student_id' column
my_students = pd.merge(my_flat_1, my_flat_corses, how='inner', on=['student_id'])

## see the DF cocntent
my_students




{"student_id":{"0":"1","1":"2"},"student_name":{"0":"Adrian","1":"Jim"},"course":{"0":["Math","Chemistry"],"1":["Math","Sports"]},"contact_mobile":{"0":12345,"1":999999},"contact_home":{"0":54321,"1":88888}}


## You all follow very well until now - but now let`s get serious :) 

In [353]:

import json 


### Load the content of the json file into my_data object
with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\nested\nested_04.json') as f:
    my_data = json.load(f)

### print it`s contents 
# print(dumps(my_data, indent=2))

# {
#   "destination_addresses":[
#     "Philadelphia, PA, USA"
#   ],
#   "origin_addresses":[
#     "New York, NY, USA"
#   ],
#   "rows":[
#     {
#       "elements":[
#         {
#           "distance":{
#             "text":"94.6 mi",
#             "value":152193
#           },
#           "duration":{
#             "text":"1 hour 44 mins",
#             "value":6227
#           },
#           "status":"OK"
#         }
#       ]
#     }
#   ],
#   "status":"OK"
# }

# pd.json_normalize(my_data)

## Ok so we can see that onlt one column was flatten, the status one ... not very felpfull tho

## lets get to work 
## Lets get the rows data out 
pd.json_normalize(data=my_data,
                  record_path=['rows','elements'])

## But we are still missing some data like : destination_addresses, origin_addresses and status 
## we will have to use the meta parameter to get this three columns from the top level 
# pd.json_normalize(data=my_data,
#                   record_path=['rows','elements'],
#                   meta=['destination_addresses','origin_addresses','status'])

## We get an error 
# ValueError: Conflicting metadata name status, need distinguishing prefix 
## this is because we have two columns with the name od "status"

# we have a fix for this 
## we will use record_prefix parameter to prefix our "rows.elements" columns
pd.json_normalize(data=my_data,
                  record_path=['rows','elements'],
                  meta=['destination_addresses','origin_addresses','status'],
                  record_prefix='lvl2_')

## but still not looking good 
## lets replace the column names '.' with an underscore
pd.json_normalize(data=my_data,
                  record_path=['rows','elements'],
                  meta=['destination_addresses','origin_addresses','status'],
                  record_prefix='lvl2_',
                  sep='_')


## We did good so far - but let`s finish with bang !!! 

,lvl2_status,lvl2_distance_text,lvl2_distance_value,lvl2_duration_text,lvl2_duration_value,destination_addresses,origin_addresses,status
0,OK,94.6 mi,152193,1 hour 44 mins,6227,"Philadelphia, PA, USA","New York, NY, USA",OK


In [405]:

import json 


### Load the content of the json file into my_data object
with open(r'C:\Users\adriano\Desktop\Projects\Personal\Python Pandas\DataSources\nested\nested_05.json') as f:
    my_data = json.load(f)

### print it`s contents 
# print(dumps(my_data, indent=2))


# {
#   "destination_addresses":[
#     "Washington, DC, USA",
#     "Philadelphia, PA, USA",
#     "Santa Barbara, CA, USA",
#     "Miami, FL, USA",
#     "Austin, TX, USA",
#     "Napa County, CA, USA"
#   ],
#   "origin_addresses":[
#     "New York, NY, USA"
#   ],
#   "rows":[
#     {
#       "elements":[
#         {
#           "distance":{
#             "text":"227 mi",
#             "value":365468
#           },
#           "duration":{
#             "text":"3 hours 54 mins",
#             "value":14064
#           },
#           "status":"OK"
#         },
#         {
#           "distance":{
#             "text":"94.6 mi",
#             "value":152193
#           },
#           "duration":{
#             "text":"1 hour 44 mins",
#             "value":6227
#           },
#           "status":"OK"
#         },
#         {
#           "distance":{
#             "text":"2,878 mi",
#             "value":4632197
#           },
#           "duration":{
#             "text":"1 day 18 hours",
#             "value":151772
#           },
#           "status":"OK"
#         },
#         {
#           "distance":{
#             "text":"1,286 mi",
#             "value":2069031
#           },
#           "duration":{
#             "text":"18 hours 43 mins",
#             "value":67405
#           },
#           "status":"OK"
#         },
#         {
#           "distance":{
#             "text":"1,742 mi",
#             "value":2802972
#           },
#           "duration":{
#             "text":"1 day 2 hours",
#             "value":93070
#           },
#           "status":"OK"
#         },
#         {
#           "distance":{
#             "text":"2,871 mi",
#             "value":4620514
#           },
#           "duration":{
#             "text":"1 day 18 hours",
#             "value":152913
#           },
#           "status":"OK"
#         }
#       ]
#     }
#   ],
#   "status":"OK"
# }


## It is a similar json file as the previous one 
## So, just run the snipet from before 

# pd.json_normalize(data=my_data,
#                   record_path=['rows','elements'],
#                   meta=['destination_addresses','origin_addresses','status'],
#                   record_prefix='lvl2_',
#                   sep='_')

# ValueError: Length of values does not match length of index
## Ups ... yeah not what we want ! 

## Let`s get to work ! 
pd.json_normalize(data=my_data)


## Now we have destination_addresses and rows beeing nested 

destination_addresses_df = pd.json_normalize(data=my_data,
                  record_path=['destination_addresses'],
                  meta=['origin_addresses','status'],
                  record_prefix='destination_addresses',
                  sep='_')
## Change the name of the column - simple estetics
destination_addresses_df.rename(columns = {'destination_addresses0':'destination_addresses'}, inplace=True)

## view the DF
destination_addresses_df


## Create the DF to include the rows elements
## you can see that our root will be rows.elements and we will prefix it with rows, plus we 
## will chage the sepparator from '.' to '_', it look better
rows_df = pd.json_normalize(data=my_data,
                  record_path=['rows','elements'],
                  meta=['status'],
                  record_prefix='rows_',
                  sep='_')
rows_df

## we are going to join the DF`s using concat and our key will be the axis=1 
my_final = pd.concat([destination_addresses_df, rows_df], axis=1)

## Finaly we wanna switch the order of some columns to make it readable
my_final = my_final[['origin_addresses','destination_addresses',
       'rows_distance_text', 'rows_distance_value', 'rows_duration_text',
       'rows_duration_value', 'rows_status']] 

my_final

,origin_addresses,destination_addresses,rows_distance_text,rows_distance_value,rows_duration_text,rows_duration_value,rows_status
0,"New York, NY, USA","Washington, DC, USA",227 mi,365468,3 hours 54 mins,14064,OK
1,"New York, NY, USA","Philadelphia, PA, USA",94.6 mi,152193,1 hour 44 mins,6227,OK
2,"New York, NY, USA","Santa Barbara, CA, USA","2,878 mi",4632197,1 day 18 hours,151772,OK
3,"New York, NY, USA","Miami, FL, USA","1,286 mi",2069031,18 hours 43 mins,67405,OK
4,"New York, NY, USA","Austin, TX, USA","1,742 mi",2802972,1 day 2 hours,93070,OK
5,"New York, NY, USA","Napa County, CA, USA","2,871 mi",4620514,1 day 18 hours,152913,OK
